In [ ]:
# 1 Generation: The model produces an initial draft of the solution.
# Critique & Refinement: The model takes its own first draft as input, critiques it against a set of criteria, and then generates a superior, improved version.

In [1]:
import litellm
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    if "gpt-5" in model:
        kwargs["max_completion_tokens"] = max_tokens
    else:
        kwargs["max_tokens"] = max_tokens
        
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        **kwargs
    )

    return response.choices[0].message.content

print("Setup complete. Helper functions and code context are ready.")

Setup complete. Helper functions and code context are ready.


In [2]:
# The First Draft (Generation)

prompt_v1 = [
    {
        "role": "user",
        "content": dedent("""
        Write a Python function named `validate_email` that takes and email string and
        returns True if valid, False otherwise. Output only the code.
        """)
    }
]

first_draft_code = get_completion(prompt_v1)

print(first_draft_code)

assistant_response_v1 = [
    {
        "role": "assistant",
        "content": first_draft_code
    }
]

```python
import re

def validate_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(pattern, email) is not None
```


In [3]:
# The Second Draft (Critique & Refinement)

prompt_refinement = [
    {
        "role": "user",
        "content": dedent("""
        Given the information and previous exchanges, please perform the following two actions:

        1. Critique: Identify specific issues with the proposed solution, considering the solution domain.
        2. Refine: Provide an improved version that addresses these issues. Keep it more robust and
            production-ready.
        """)
    }
]

final_conversation = prompt_v1 + assistant_response_v1 + prompt_refinement

revised_code = get_completion(final_conversation)

display(Markdown(revised_code))

### Critique

1. **Regex Complexity**: The regular expression used for email validation is not fully compliant with the official specifications for email formats (RFC 5321 and RFC 5322). It may incorrectly accept or reject valid email addresses and allows some potentially invalid formats.

2. **Internationalization**: The current solution does not handle internationalized domain names or email addresses (e.g., those containing non-ASCII characters).

3. **Length Limitations**: There are size limits on the total length of an email address (generally 254 characters), which is not enforced in the current implementation.

4. **Performance on Large Inputs**: The use of regex may lead to performance issues, especially with larger input strings, due to backtracking.

5. **Lack of Clear Reporting Mechanism**: The function only returns `True` or `False`, making it difficult to understand why an email might be considered invalid.

### Refine

Here’s an improved version addressing these issues. 

```python
import re

def validate_email(email):
    if not isinstance(email, str):
        return False
    
    # Check length
    if len(email) > 254:
        return False

    # Validating with a more robust regex pattern
    pattern = r'^(?=.{1,254}$)(?=.{1,64}@.{1,255}$)[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    
    # Check regex match
    if not re.match(pattern, email):
        return False
    
    return True
```

### Improvements Made:

1. **Length Check**: Added a check to ensure the total length does not exceed 254 characters.

2. **Improved Regex**: The new regex pattern adds conditions to ensure compliance with length restrictions for both the local part and the domain part.

3. **Type Check**: Added a check to ensure the input is a string.

4. **Enhanced Validation**: The regex is more closely aligned with valid email formats while still being easy to read.

This refined version provides a more comprehensive and robust solution for email validation.